In [1]:
import trading as trd

Using Theano backend.


In [2]:
# relaod module after modifying it
import importlib
importlib.reload(trd.stock_history)
importlib.reload(trd.nn)
importlib.reload(trd.testing)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\school\\cs_229\\cs229_final_project\\src\\trading\\__init__.py'>

## Portfolio Setup

In [3]:
# rebalance frequency
rebalance_period = 30

In [4]:
model_dirs = ['2days_pen_return_l=0_5_100h', '2days_pen_return_l=0_100h',
              '2days_sharpe_reward_100h', '7days_pen_return_l=0_5_100h',
              '7days_pen_return_l=0_100h', '7days_sharpe_reward_100h']

In [5]:
trd.test_models(model_dirs)

running model 2days_pen_return_l=0_5_100h (1 of 6)
running model 2days_pen_return_l=0_100h (2 of 6)
running model 2days_sharpe_reward_100h (3 of 6)
running model 7days_pen_return_l=0_5_100h (4 of 6)
running model 7days_pen_return_l=0_100h (5 of 6)
running model 7days_sharpe_reward_100h (6 of 6)
